# Property-based Testing mit Hypothesis

Simon Jakobi (https://github.com/sjakobi)

http://hypothesis.readthedocs.org

## Properties

* Hält meine Funktion für alle gültigen Inputs einen ordentlichen Output aus?
* gilt `decode(encode(x)) == x`?

In [257]:
from hypothesis import given
import hypothesis.strategies as st

@given(st.integers())
def test_int_str_conversion(n):
    assert int(str(n)) == n
    
test_int_str_conversion()

In [258]:
from hypothesis import Settings, assume, Verbosity, example
from math import isnan

@example(-0.0)
@given(st.floats(), settings=Settings(verbosity=Verbosity.verbose))
def test_float_str_conversion(n):
    assume(not isnan(n))
    assert float(str(n)) == n
    
test_float_str_conversion()

Trying example: test_float_str_conversion(n=nan)
Trying example: test_float_str_conversion(n=3.6747825971579628e+264)
Trying example: test_float_str_conversion(n=5.813125574739855e+144)
Trying example: test_float_str_conversion(n=-2.2250738585072014e-308)
Trying example: test_float_str_conversion(n=1360.0)
Trying example: test_float_str_conversion(n=466.0)
Trying example: test_float_str_conversion(n=542341094604065.9)
Trying example: test_float_str_conversion(n=863.0)
Trying example: test_float_str_conversion(n=2.072475237841613e-309)
Trying example: test_float_str_conversion(n=706.0)
Trying example: test_float_str_conversion(n=2786.0)
Trying example: test_float_str_conversion(n=-470.0)
Trying example: test_float_str_conversion(n=1.3936899804763441e+28)
Trying example: test_float_str_conversion(n=290.0)
Trying example: test_float_str_conversion(n=1.2160852373662958)
Trying example: test_float_str_conversion(n=1.1190726712849837)
Trying example: test_float_str_conversion(n=1.13296837564

In [153]:
def quicksort(lst):
    if not lst:
        return []
    lst_ = lst[:]
    x = lst_.pop()
    return (quicksort([y for y in lst_ if y <= x]) +
            [x] +
            quicksort([y for y in lst_ if y >= x]))

Idempotenz: `sorted(sorted(xs)) == sorted(xs)`

In [259]:
@given(st.lists(st.integers(), max_size=3)) # settings=Settings(verbosity=Verbosity.verbose))
def test_quicksort_is_idempotent(lst):
    sorted_ = quicksort(lst)
    assert quicksort(sorted_) == sorted_
    
test_quicksort_is_idempotent()

Falsifying example: test_quicksort_is_idempotent(lst=[0, 0])


AssertionError: 

### Ist der Output von quicksort wirklich sortiert?

In [260]:
# Erster Versuch (geschönt)

MAX_SIZE = 10
lists = st.lists(st.integers(), max_size=MAX_SIZE)
indices = st.integers(min_value=0, max_value=MAX_SIZE)
index_pairs = st.tuples(indices, indices)

@given(lists, index_pairs)
def test_quicksort_result_is_sorted(lst, index_pair):
    assume(index_pair[0] < len(lst))
    assume(index_pair[1] < len(lst))
    idx1, idx2 = sorted(index_pair)
    sorted_lst = quicksort(lst)
    assert sorted_lst[idx1] <= sorted_lst[idx2]
    
test_quicksort_result_is_sorted()

Addendum:
    
Im Nachhinein erscheint es ein wenig overkill, die Indizes mit Hypothesis zu generieren.
Einfacher geht es so:

In [262]:
@given(lists)
def test_quicksort_result_is_sorted2(lst):
    sorted_lst = quicksort(lst)
    for (x, y) in zip(sorted_lst, sorted_lst[1:]):
        assert x <= y

test_quicksort_result_is_sorted2()

Am einfachsten: Vergleich mit einer Referenzimplementierung

In [162]:
@given(st.lists(st.integers()))
def test_compare_to_sorted(lst):
    assert quicksort(lst) == sorted(lst)
    
test_compare_to_sorted()

Falsifying example: test_compare_to_sorted(lst=[0, 0])


AssertionError: 

## Strategies

In [8]:
help(st.integers)

Help on function integers in module hypothesis.strategies:

integers(min_value=None, max_value=None)
    Returns a strategy which generates integers (in Python 2 these may be
    ints or longs).
    
    If min_value is not None then all values will be >=
    min_value. If max_value is not None then all values will be <= max_value



### Strategien transformieren

In [33]:
st.integers().map(lambda n: 2 * n).example()

-16

### Strategien kombinieren

In [264]:
strat = st.one_of(st.integers(), st.floats())

[strat.example() for _ in range(6)]

[-3,
 -7.026538184441703e-309,
 1.0674221941712151e-69,
 -87.0,
 -4,
 835276951908890046543449252112211892375994387932703844421397285087355441913]

In [132]:
help(st.lists)

Help on function lists in module hypothesis.strategies:

lists(elements=None, min_size=None, average_size=None, max_size=None, unique_by=None)
    Returns a list containining values drawn from elements length in the
    interval [min_size, max_size] (no bounds in that direction if these are
    None). If max_size is 0 then elements may be None and only the empty list
    will be drawn.
    
    average_size may be used as a size hint to roughly control the size
    of list but it may not be the actual average of sizes you get, due
    to a variety of factors.
    
    if unique_by is not None it must be a function returning a hashable type
    when given a value drawn from elements. The resulting list will satisfy the
    condition that for i != j, unique_by(result[i]) != unique_by(result[j]).



In [133]:
help(st.recursive)

Help on function recursive in module hypothesis.strategies:

recursive(base, extend, max_leaves=100)
    base: A strategy to start from
    extend: A function which takes a strategy and returns a new strategy
    max_leaves: The maximum number of elements to be drawn from base on a given
    run.
    
    This returns a strategy S such that S = extend(base | S). That is, values
    maybe drawn from base, or from any strategy reachable by mixing
    applications of | and extend.
    
    An example may clarify: recursive(booleans(), lists) would return a
    strategy that may return arbitrarily nested and mixed lists of booleans.
    So e.g. False, [True], [False, []], [[[[True]]]], are all valid values to
    be drawn from that strategy.



## Eigene Datenstrukturen

In [267]:
from collections import namedtuple
from math import sqrt

from hypothesis import Settings, Verbosity

Point = namedtuple("Point", "x, y")

def distance(p1, p2):
    dx = (p1.x - p2.x) ** 2
    dy = (p1.y - p2.y) ** 2
    return sqrt(dx + dy)

In [171]:
# Ein konventioneller Test für die Plausibilität
def test_distance_from_0_0_to_0_1_is_1():
    assert distance(Point(0,0), Point(0,1)) == 1

test_distance_from_0_0_to_0_1_is_1()

### Mit Integer-Koordinaten

In [277]:
points = st.builds(Point, x=st.integers(), y=st.integers())

In [270]:
@given(points)
def test_distance_for_same_point_is_zero(point):
    assert distance(point, point) == 0
    
test_distance_for_same_point_is_zero()

In [271]:
@given(points, points)
def test_distance_is_always_non_negative(p1, p2):
    assert distance(p1, p2) >= 0
    
test_distance_is_always_non_negative()

In [278]:
@given(points, points)
def test_distance_is_commutative(p1, p2):
    assert distance(p1, p2) == distance(p2, p1)
    
test_distance_is_commutative()

### Mit Float-Koordinaten

In [282]:
points = st.builds(Point, x=st.floats(), y=st.floats())

In [276]:
@given(points)
def test_distance_for_same_point_is_zero(point):
    assert distance(point, point) == 0
    
test_distance_for_same_point_is_zero()

Falsifying example: test_distance_for_same_point_is_zero(point=Point(x=0.0, y=inf))


AssertionError: 

### Ohne Inf, -Inf, NaN

In [285]:
from math import isfinite

finite_floats = st.floats().filter(isfinite)

[finite_floats.example() for _ in range(5)]

[-9.80239181252283e-113,
 32.87147634543405,
 8.261931442927766e-309,
 0.0,
 2.601102708210644e-270]

In [286]:
points = st.builds(Point, finite_floats, finite_floats)

In [287]:
@given(points)
def test_distance_for_same_point_is_zero(point):
    assert distance(point, point) == 0
    
test_distance_for_same_point_is_zero()

### Können die eingebauten Strategien alle Problemstellen finden?

In [292]:
def hyperbel_mit_unstetigkeit(x):
    def hyperbel(n):
        return 1 / (n - x)
    return hyperbel

In [301]:
@given(st.floats())
def test_hyperbel(n):
    hyperbel = hyperbel_mit_unstetigkeit(5.0)
    assert isinstance(hyperbel(n), object)
    
test_hyperbel()

Falsifying example: test_hyperbel(n=5.0)


ZeroDivisionError: float division by zero

In [302]:
from math import pi

@given(st.floats())
def test_hyperbel(n):
    hyperbel = hyperbel_mit_unstetigkeit(pi)
    assert isinstance(hyperbel(n), object)
    
test_hyperbel()

In [303]:
# Nebenbemerkung

pi + (-pi) == 0

True

## Strategien verknüpfen

In [306]:
help(st.integers().flatmap) # ???

Help on method flatmap in module hypothesis.searchstrategy.strategies:

flatmap(expand) method of hypothesis.searchstrategy.reprwrapper.ReprWrapperStrategy instance
    Returns a new strategy that generates values by generating a value
    from this strategy, say x, then generating a value from
    strategy(expand(x))
    
    This method is part of the public API.



### Beispiel: Listen der Länge n mit Elementen aus [0..n]

In [331]:
small_ints = st.integers(min_value=0, max_value=10)
special_lists = small_ints.flatmap(
    lambda n: st.lists(st.integers(min_value=0, max_value=n), min_size=n, max_size=n)
)
[special_lists.example() for _ in range(5)]

[[0, 2],
 [0, 5, 1, 4, 2],
 [10, 3, 9, 4, 8, 4, 0, 3, 7, 9],
 [0],
 [4, 4, 6, 6, 5, 5, 5]]

## Strings

In [335]:
from pprint import pprint

strat = st.text()
pprint([strat.example() for _ in range(5)])

['ă^ăr1^\n\x89ă\x891^ă\n\n\x89\x89\x9eă',
 'ǐm\U000f383f Z\x05흨ǐ\U00075029',
 'I\x01',
 '\x82\x84\t\x1b\x1e',
 '\r\x85\x0c\x87']


## Offene Intervalle

In [344]:
def open_interval(lower_endpoint=None, upper_endpoint=None):
    return st.floats(min_value=lower_endpoint, max_value=upper_endpoint).filter(
        lambda n: n not in (lower_endpoint, upper_endpoint)
    )

[open_interval(0, 1).example() for _ in range(5)]

[0.16412421076454267,
 0.40942398612599684,
 0.8156785897510119,
 0.5,
 0.5689206788828095]

## Ziehen von zufälligen Elementen aus einer vorgegebenen Menge

In [346]:
from string import ascii_lowercase

random_lowercase_letter = st.sampled_from(ascii_lowercase)

[random_lowercase_letter.example() for _ in range(5)]

['y', 'n', 'z', 'x', 'h']

## Vorteile von Property-based Testing

* Kann Sonderfälle finden, auf die man selbst nicht so schnell gekommen wäre
* Viele Testcases mit wenig Code
* Regt an, über Spezifikationen nachzudenken

## Nachteile von Property-based Testing

* Geeignete Properties sind nicht immer leicht zu finden
* Testdurchläufe können lange dauern
* Tests können kompliziert werden, gar selbst testbedürftig werden

## Weiterführendes

Apis testen:
    
* http://wildfish.com/blog/2015/10/01/using-gabbi-and-hypothesis-test-django-apis/
* http://hypothesis.readthedocs.org/en/latest/examples.html#fuzzing-an-http-api

Eine Testsuite vom Hypothesis-Autor selbst:

https://github.com/DRMacIver/intset/blob/master/tests/test_intset.py